<a href="https://colab.research.google.com/github/kovzanok/ml/blob/main/hw_01_logistic_regression_for_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание. Логистическая регрессия для классификации текста
В этом домашнем задании вам предстоит решить каноническую задачу классификации текста --- создать спам-фильтр. Задание большей частью вам придется решать самостоятельно. Для этого можно использовать методы, которые мы проходили на семинаре.

## Данные и посылка
Как и во многих следующих домашних заданиях курса, в этом задании мы будем проверять и оценивать вашу посылку так, как это происходит на конкурсах по машинному обучению (например, на платформе kaggle.com). Для этого мы предварительно разделили данные в задании на две части.

* `data_train.csv`. В каждой строчке файла `data_train.csv` указан текст, который необходимо классифицировать, (столбец `Message`) и целевая переменная (столбец `Category`), принимающая 2 значения: "spam" и "ham", где spam --- это спамовое сообщение, а ham --- просто sms-ка. На этом наборе данных вам необходимо создать и обучить модель.
* `data_scoring.csv`. В каждой строчке файла `data_scoring.csv` указан текст, который необходимо классифицировать, используя обученную модель. Полученные предсказания необходимо записать в файл `submission.csv` и сдать в соответствующий шаг на Stepik. Количество попыток ограничено 100 штук.

Также мы предоставляем файл [`sample_submission.csv`](https://drive.google.com/file/d/1uGPbOSiZx0S9dTdQnam16ncXE6w0JvFm/view?usp=share_link). Это образец посылки --- ваш файл `submission.csv` должен быть оформлен таким же образом.

## Целевая метрика и выставление баллов
Целевая метрика в этой задаче --- ROC-AUC. Именно по ней система будет выставлять вам баллы. Минимальное значение ROC-AUC, за которое ставятся баллы, равно $lb=0.90$. Чтобы набрать полный балл за задание, необходимо набрать $ub=0.99$.
Оценка выставляется по формуле
$$\begin{cases}
0, & score \leqslant lb,\\
10 \cdot \frac{score - lb}{ub - lb}, & lb < score < ub,\\
10, & score \geqslant ub.
\end{cases}$$

Обратите внимание: авторы курса не гарантируют, что качество $0.99$ можно получить методами с семинара. Наше решение дает качество 0.974, но скорее всего аккуратным перебором гиперпараметров можно получить баллы выше.

## Ход работы
Мы предлагаем следующий пайплайн работы.
* Загрузка и обработка данных `data_train.csv`. Часть этой загрузки мы сделали за вас. На этом этапе рекомендуем воспользоваться схемой из семинара.
* Обучение первого варианта модели. Выберите любую модель, которую мы рассматривали на семинаре, и посмотрите, какое качество она дает на наших данных (для этого внутри этого пункта выделите небольшую тестовую выборку, в следующих пунктах она вам не пригодится, так как там мы будем использовать кросс-валидацию).
* Написание пайплайна тестирования. Специфика данных состоит в том, что размер `data_train` мал по сравнению с размером `data_scoring`. Поэтому разделять данные в `data_train` на обучающее и тестирующее множество жалко. По этой причине мы рекомендуем вам воспользоваться процедурой кросс-валидации для более надеждного измерения качества модели. Вам поможет `sklearn.model_selection.cross_validate`.
* Обратите внимание, что если при кросс-валидации кодировать с помощью Bag of Words данные на всем обучающем множестве $X$, то это повлечет переобучение (поскольку, например, модель, обученная на фолдах №1, №2, №3, №4, будет иметь информацию о словах, которые встречались в текстах фолда №5, но не будет иметь информацию о словах из `data_scoring`). Поэтому мы рекомендуем вам реализовать кросс-валидацию вручную, для каждого фолда обучая свой Bag Of Words: тогда качество на каждом фолде будет "честным". Для того, чтобы реализовать кросс-валидацию вручную, можно использовать [`sklearn.model_selection.KFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html).
Впрочем, хорошие результаты получаются и при использовании стандартной кросс-валидации (но качество на `data_scoring` при сдаче в систему в таком случае может немного упасть).
* Загрузка данных из `data_scoring.csv`, применение модели и создание `submission.csv`. **Обязательно проверьте, что формат файла совпадает с форматом `sample_submission.csv`!**

Желаем удачи!

In [44]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder
import nltk
%matplotlib inline

## Загрузка данных

In [2]:
!gdown 1yacgNpDCFREwRe0qiOpTyoik2Jz_z-fu  # data_train.csv
!gdown 1yt9p3Syey65CnNJugcFZsNrOGELBd3LS  # data_scoring.csv

Downloading...
From: https://drive.google.com/uc?id=1yacgNpDCFREwRe0qiOpTyoik2Jz_z-fu
To: /content/data_train.csv
100% 34.7k/34.7k [00:00<00:00, 57.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yt9p3Syey65CnNJugcFZsNrOGELBd3LS
To: /content/data_scoring.csv
100% 424k/424k [00:00<00:00, 103MB/s]


In [3]:
data = pd.read_csv('data_train.csv')
data.head()

,Category,Message
0,ham,TODAY is Sorry day.! If ever i was angry with ...
1,ham,I.ll give her once i have it. Plus she said gr...
2,ham,Where is it. Is there any opening for mca.
3,ham,K:)k:)what are detail you want to transfer?acc...
4,ham,"Wen ur lovable bcums angry wid u, dnt take it ..."


<Axes: >

In [4]:
data['Category'].hist()

**Задание**. Обработайте данные: выделите целевую переменную, произведите лемматизацию (по желанию).

In [13]:
encoder = LabelEncoder()

encoded_labels = encoder.fit_transform(data['Category'])
data['y'] = encoded_labels

In [29]:
nltk.download('punkt_tab')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize


tokens = [list(filter(lambda s:s.isalpha(), word_tokenize(text.lower()))) for text in data['Message'].to_list()]

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

text = []

for text_tokens in tokens:
    lemmatized_message = []
    for word in text_tokens:
        lemmatized_word = lemmatizer.lemmatize(word)
        lemmatized_message.append(word)
    text.append(' '.join(lemmatized_message))

In [35]:
data['x'] = text

In [37]:
data.head()

,Category,Message,y,x
0,ham,TODAY is Sorry day.! If ever i was angry with ...,0,today is sorry if ever i was angry with you if...
1,ham,I.ll give her once i have it. Plus she said gr...,0,give her once i have it plus she said grinule ...
2,ham,Where is it. Is there any opening for mca.,0,where is it is there any opening for mca
3,ham,K:)k:)what are detail you want to transfer?acc...,0,k k what are detail you want to transfer acc n...
4,ham,"Wen ur lovable bcums angry wid u, dnt take it ...",0,wen ur lovable bcums angry wid u dnt take it s...


## Обучение первого варианта модели
**Задание.** Выделите (только для этого пункта) из data_train тестовую часть. Обучите какую-то из моделей, которые мы использовали на семинаре, и посчитайте качество на тестовых данных.

In [87]:
X_train, X_test, y_train, y_test = train_test_split(data['x'], data['y'], train_size=0.8)

In [88]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

vectorizer = CountVectorizer(stop_words=list(stop_words))
vectorizer.fit(X_train)

vocabulary = vectorizer.get_feature_names_out()
description_count = vectorizer.transform(X_train)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [89]:
def get_top_k_words(vocabulary, description_count, k = 10):
    top_k = sorted(zip(vocabulary, description_count.sum(axis=0).getA1()), key=lambda tup:tup[1], reverse=True)[:k]
    return ', '.join([word for word, _ in top_k])

In [90]:
get_top_k_words(vocabulary, description_count)

'get, call, ur, love, come, go, lt, need, gt, ok'

In [91]:
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

## Написание пайплайна тестирования
Задание. Напишите схему кросс-валидации, которая будет тестировать качество модели с данными параметрами.

## Выбор лучшей модели
Протестируйте с помощью написанного пайплайна несколько моделей, которые мы рассматривали на семинаре. Выберите ту из них, которая дает лучшее среднее качество на кросс-валидации.

## Обучение финальной модели
После выбора лучшей модели, обучите ее на всех доступных данных (`data_train`).

In [ ]:
best_model = #YOUR CODE

LogisticRegression()

## Применение модели на data_scoring и формирование submission
Загрузите данные из `data_scoring.csv`. Обработайте их так же, как вы обработали `data_train.csv`. Примените лучшую модель и сформируйте submission. Сдайте ее на Stepik.

**Обратите внимание, что метрика, используемая в задании --- ROC-AUC. Поэтому вам необходимо предсказать не класс, а вероятность класса `spam`.**

In [ ]:
data_scoring = pd.read_csv('data_scoring.csv')
data_scoring.head()

,Message
0,"Hurry up, I've been weed-deficient for like th..."
1,Sounds like a plan! Cardiff is still here and ...
2,Is it your yahoo boys that bring in the perf? ...
3,"K, I'll work something out"
4,Hi im having the most relaxing time ever! we h...


In [ ]:
#YOUR CODE

In [ ]:
y_scoring = # YOUR CODE. Сюда запишите вероятности принадлежности к
# классу "spam" полученные с помощью модели на data_scoring.csv

In [ ]:
pd.Series(y_scoring, name='score').to_csv('submission.csv', index=None)